# Importing libraries

In [1]:
import os
import shutil
import requests
import time
from bs4 import BeautifulSoup
import numpy as np
from pdf2image import convert_from_path
import cv2
import pytesseract
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Web scrapping to download the required pdf files

In [ ]:
# Define the base URL of the SEC folder containing PDFs
base_url = "https://www.sec.gov/Archives/edgar/vprr/"

# Define headers as per SEC's guidelines
headers = {
    "User-Agent": "adishinde2110@gmail.com",
    "Accept-Encoding": "gzip, deflate",
    "Host": "www.sec.gov"
}

# Define a function to download a PDF file
def download_pdf(pdf_url, folder_name):
    try:
        response = requests.get(pdf_url, headers=headers)
        response.raise_for_status()

        # Create a directory if it doesn't exist
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)

        file_name = os.path.join(folder_name, pdf_url.split("/")[-1])
        with open(file_name, "wb") as pdf_file:
            pdf_file.write(response.content)

        print(f"Downloaded: {file_name}")

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")

# List of parent directories
parent_dirs = ['0004', '0006', '0009', '0010', '0011', '0100', '0101',
               '0102', '0103','0104', '0105', '0106', '0107', '0108']

# Create a folder called "forms" before creating the folder based on the last subdirectory of the URL
base_folder_name = "forms_raw"

# Loop through the parent directories
for parent_dir in parent_dirs:
    # Construct the full base URL for the current parent directory
    current_base_url = base_url + parent_dir + "/"

    # Send a request to the current base URL
    response = requests.get(current_base_url, headers=headers)
    response.raise_for_status()

    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, "html.parser")

    # Find all links to PDF files in the folder
    pdf_links = [link.get("href") for link in soup.find_all("a") if link.get("href").endswith(".pdf")]

    # Extract the last subdirectory of the URL to create a folder name
    folder_name = current_base_url.rstrip("/").split("/")[-1]

    # Combine the base folder name "forms" and the last subdirectory folder name
    folder_name = os.path.join(base_folder_name, folder_name)

    # Download the PDFs while respecting the request rate limit (10 requests per second)
    for pdf_link in pdf_links:
        pdf_url = current_base_url + pdf_link.rstrip("/").split("/")[-1]
        download_pdf(pdf_url, folder_name)

        # Pause to respect the request rate limit (10 requests per second)
        time.sleep(0.1)

# Converting pdf to image for OCR

In [2]:
# Specify the root directory where your "forms" folder is located
root_directory = './forms/'

# Function to convert the first page of PDFs to images in a folder and delete the PDFs
def convert_first_page_to_images_and_delete(folder_path):
    # List all PDF files in the folder
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]

    for pdf_file in pdf_files:
        pdf_path = os.path.join(folder_path, pdf_file)
        images = convert_from_path(pdf_path, first_page=0, last_page=1, poppler_path='C:/Program Files/poppler-23.08.0/Library/bin')

        if images:
            # Save the first page image (index 0) as a JPEG file
            first_page_image = images[0]
            image_path = os.path.join(folder_path, f'{pdf_file}_page1.jpg')
            first_page_image.save(image_path, 'JPEG')

        # Delete the PDF file
        os.remove(pdf_path)

# Iterate through subdirectories of the root directory
for folder_name in os.listdir(root_directory):
    folder_path = os.path.join(root_directory, folder_name)

    if os.path.isdir(folder_path):
        convert_first_page_to_images_and_delete(folder_path)

# Creating balanced dataset for all document types

In [3]:
source_folder = './forms/'

for folder in os.listdir(source_folder):
    folder_path = os.path.join(source_folder, folder)
    if os.path.isdir(folder_path):
        num_images = len(os.listdir(folder_path))
        if num_images < 200:
            images_to_duplicate = 200 - num_images
            images_in_folder = os.listdir(folder_path)
            for i in range(images_to_duplicate):
                image_to_copy = images_in_folder[i % num_images]  # Cycle through existing images
                source_image_path = os.path.join(folder_path, image_to_copy)
                new_image_name = f'duplicated_{i}.jpg'
                new_image_path = os.path.join(folder_path, new_image_name)
                shutil.copy(source_image_path, new_image_path)

# OCR and text pre-processing

In [4]:
def preprocess_text(text):
    stopwords = ["0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz"]

    # Tokenize the text by splitting it into words
    words = text.split()
    
    # Remove stopwords
    filtered_words = [word for word in words if word.lower() not in stopwords]
    
    # Join the filtered words back into a single string
    processed_text = ' '.join(filtered_words)
    
    return processed_text


# Step 1: Preprocess the images and create the dataset
data = []
labels = []

# Specify the root directory containing subdirectories for each class
root_dir = "./forms/"

for folder_name in os.listdir(root_dir):
    folder_path = os.path.join(root_dir, folder_name)
    
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        # Use OpenCV to load and preprocess the image
        image = cv2.imread(file_path)
        grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Use Tesseract OCR to extract text
        extracted_text = pytesseract.image_to_string(grayscale_image)
        
        # Clean and preprocess the extracted text
        cleaned_text = preprocess_text(extracted_text)

        data.append(cleaned_text)
        labels.append(folder_name)

# TF-IDF vectorizer and word ranking

In [12]:
# Step 2: Create TF-IDF vectors
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust the number of features as needed
tfidf_matrix = tfidf_vectorizer.fit_transform(data)

# Get the feature names (words)
feature_names = tfidf_vectorizer.get_feature_names_out()

# Sum the TF-IDF scores for each feature (word) across all documents
tfidf_scores = tfidf_matrix.sum(axis=0)

# Create a dictionary that maps feature names to their TF-IDF scores
feature_tfidf_scores = {}
for feature, score in zip(feature_names, tfidf_scores.tolist()[0]):
    feature_tfidf_scores[feature] = score

# Sort the features by their TF-IDF scores in descending order
sorted_features = sorted(feature_tfidf_scores.items(), key=lambda x: x[1], reverse=True)

# Get the top N most frequent words (features)
top_n = 100  # You can adjust this to get the desired number of top words
top_features = sorted_features[:top_n]

# Print the top N most frequent words with word rank
for rank, (feature, score) in enumerate(top_features, start=1):
    print(f"Rank: {rank}, {feature}, TF-IDF Score: {score}")

Rank: 1, securities, TF-IDF Score: 150.79066267028057
Rank: 2, report, TF-IDF Score: 120.68742841136331
Rank: 3, exchange, TF-IDF Score: 110.03243692359685
Rank: 4, form, TF-IDF Score: 108.24084516057495
Rank: 5, address, TF-IDF Score: 105.98944237295144
Rank: 6, plan, TF-IDF Score: 98.06152678635914
Rank: 7, registrant, TF-IDF Score: 86.82986475545856
Rank: 8, transfer, TF-IDF Score: 86.59436681823345
Rank: 9, accountant, TF-IDF Score: 86.48696796025334
Rank: 10, notice, TF-IDF Score: 85.22003913933457
Rank: 11, file, TF-IDF Score: 83.04552122006709
Rank: 12, derivative, TF-IDF Score: 78.15262875703407
Rank: 13, product, TF-IDF Score: 76.20767094130429
Rank: 14, filing, TF-IDF Score: 73.98710064171973
Rank: 15, manager, TF-IDF Score: 73.92702676352366
Rank: 16, number, TF-IDF Score: 72.59059791312082
Rank: 17, pursuant, TF-IDF Score: 70.69489533413363
Rank: 18, no, TF-IDF Score: 70.51013813145148
Rank: 19, agent, TF-IDF Score: 70.2200848160566
Rank: 20, state, TF-IDF Score: 68.8839150

# Training and Evaluating Classification Model

In [6]:
### Step 3: Split the dataset
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, labels, test_size=0.2, random_state=42)

# Step 4: Train a classification model (e.g., SVM)
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

# Step 5: Evaluate the model
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
classification_report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(classification_report)

Accuracy: 1.0
                   precision    recall  f1-score   support

82_Submissions_FS       1.00      1.00      1.00        32
         Form_11K       1.00      1.00      1.00        47
         Form_13F       1.00      1.00      1.00        48
        Form_19B4       1.00      1.00      1.00        37
          Form_6K       1.00      1.00      1.00        36
           Form_D       1.00      1.00      1.00        32
         Form_TA2       1.00      1.00      1.00        46
       Form_X17A5       1.00      1.00      1.00        41
           Others       1.00      1.00      1.00        41

         accuracy                           1.00       360
        macro avg       1.00      1.00      1.00       360
     weighted avg       1.00      1.00      1.00       360

